In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn import metrics
import numpy as npw
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler





# Read the CSV file
data = pd.read_csv("../data/data/aapl_raw_data.csv")
data.shape
data.tail(1)

data = data.iloc[:10747]

data.tail(1)


data.isnull().sum()
data=data.fillna(0)  # Filling null values with zero
data.isnull().sum()



#data["date"] = data["date"].astype(float)
data["open"] = data["open"].astype(float)
data["high"] = data["high"].astype(float)
data["low"] = data["low"].astype(float)
data["volume"] = data["volume"].astype(float)
data["adjusted_close"] = data["adjusted_close"].astype(float)
data["change_percent"] = data["change_percent"].astype(float)
data["avg_vol_20d"] = data["avg_vol_20d"].astype(float)

data["close"] = data["close"].astype(float)

#print(data.tail(1))

In [2]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert data to numpy arrays
x_data = data[['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']].values
y_data = data["close"].values.reshape(-1, 1)  # Reshape y_data if it's 1D

# Initialize the scalers
scaler_x = StandardScaler()
scaler_y = StandardScaler()

# Fit the scaler to the training data and transform both training and test data
x_data_scaled = scaler_x.fit_transform(x_data)
y_data_scaled = scaler_y.fit_transform(y_data)

# Convert scaled data to Tensors
x_feature_tensors = torch.tensor(x_data_scaled, dtype=torch.float32)
y_feature_tensors = torch.tensor(y_data_scaled, dtype=torch.float32)

# Split the training data into training and temporary sets
x_train, x_temp, y_train, y_temp = train_test_split(x_feature_tensors, y_feature_tensors, test_size=0.2, random_state=42)

# Split the temporary set into validation and test sets
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=42)

# Convert scaled labels back to numpy arrays
y_train = y_train.numpy()
y_val = y_val.numpy()
y_test = y_test.numpy()



# Now you can use x_train_scaled, x_val, x_test, y_train_scaled, y_val_scaled, y_test_scaled for your model training


In [ ]:
learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train) - window_size
batch_size = 1
input_size = 7  # Number of features
output_size = 7  # Number of features to predict

class LSTMModel7(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel7, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden




In [ ]:
# Instantiate the model
model7 = LSTMModel7(input_size, hidden_dim, n_layers, output_size)

# Choose a loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model7.parameters(), lr=learning_rate)


epochs = 1
window_size = 20

# Training loop (use model7 instead of model)
for epoch in range(epochs):
    for i in range(0, len(x_train) - window_size, batch_size):
        inputs = x_train[i:i+window_size]
        targets = y_train[i+window_size]

        # Convert inputs and targets to PyTorch tensors
        inputs = torch.tensor(inputs, dtype=torch.float32)
        targets = torch.tensor(targets, dtype=torch.float32)

        # Zero the gradients
        optimizer.zero_grad()

        # Initialize hidden state
        hidden = model7.init_hidden(batch_size)

        # Forward pass
        outputs, _ = model7(inputs.unsqueeze(0), hidden)
        loss = criterion(outputs.squeeze(), targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print loss or other metrics occasionally
        if i % batch_evaluation_frequency == 0:
            print(f'Epoch {epoch}, Batch {i}, Loss: {loss.item()}')

C:\Users\pitr7\AppData\Local\Temp\ipykernel_9756\3406692956.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32)
c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([7])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Batch 0, Loss: 0.5208550095558167
Epoch 0, Batch 10, Loss: 0.4920283854007721
Epoch 0, Batch 20, Loss: 0.01090602483600378
Epoch 0, Batch 30, Loss: 1.1893177032470703
Epoch 0, Batch 40, Loss: 2.9421582221984863
Epoch 0, Batch 50, Loss: 0.6296281218528748
Epoch 0, Batch 60, Loss: 0.22642479836940765
Epoch 0, Batch 70, Loss: 1.9359509944915771
Epoch 0, Batch 80, Loss: 0.3882392346858978
Epoch 0, Batch 90, Loss: 0.6536073684692383
Epoch 0, Batch 100, Loss: 0.45456889271736145
Epoch 0, Batch 110, Loss: 0.10324207693338394
Epoch 0, Batch 120, Loss: 0.04227430000901222
Epoch 0, Batch 130, Loss: 0.0339929424226284
Epoch 0, Batch 140, Loss: 0.0008210118976421654
Epoch 0, Batch 150, Loss: 0.17377229034900665
Epoch 0, Batch 160, Loss: 0.34998777508735657
Epoch 0, Batch 170, Loss: 3.7915265560150146
Epoch 0, Batch 180, Loss: 0.27707523107528687
Epoch 0, Batch 190, Loss: 0.2342279702425003
Epoch 0, Batch 200, Loss: 0.00788091029971838
Epoch 0, Batch 210, Loss: 11.530004501342773
Epoch 0, 

In [ ]:
# Assuming you've already trained the model (model7) with the desired parameters

# Extract the last window from the training data
last_window = x_train[-window_size:]

# Convert the last window to a PyTorch tensor
last_window_tensor = torch.tensor(last_window, dtype=torch.float32)

# Initialize hidden state
hidden = model7.init_hidden(batch_size=1)

# Forward pass to obtain the prediction for the next row
with torch.no_grad():
    prediction, _ = model7(last_window_tensor.unsqueeze(0), hidden)

# Convert the prediction to a numpy array
predicted_row = prediction.squeeze().numpy()

# Print or use the predicted row as needed
print("Predicted Row:", predicted_row)


Predicted Row: [0.02775631 0.02834398 0.02608973 0.02595106 0.02616901 0.02624355
 0.02694829]


C:\Users\pitr7\AppData\Local\Temp\ipykernel_9756\2166579918.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  last_window_tensor = torch.tensor(last_window, dtype=torch.float32)


In [ ]:
# Print the last window
print("Last Window:")
print(last_window)


Last Window:
tensor([[-5.9760e-01, -5.8771e-01, -5.9358e-01, -4.2231e-01, -4.5338e-01,
          1.8267e+00,  7.7047e-01],
        [-7.3693e-01, -7.3899e-01, -7.3335e-01, -6.7598e-01, -4.5896e-01,
          8.7407e-01, -7.6385e-01],
        [-3.6628e-01, -3.7245e-01, -3.6745e-01,  2.6655e-01, -4.1044e-01,
         -6.6144e-01,  1.4002e+00],
        [-7.3982e-01, -7.4088e-01, -7.4107e-01,  2.2458e-01, -4.5467e-01,
         -5.6518e+00,  4.3665e-01],
        [-6.9224e-01, -6.9241e-01, -6.8822e-01, -6.5042e-01, -4.5190e-01,
          5.6484e-01, -2.9350e-01],
        [-5.4170e-01, -5.3405e-01, -5.3618e-01, -6.5565e-01, -4.5247e-01,
          8.1720e-01, -4.4281e-01],
        [ 4.1231e-01,  4.2911e-01,  4.0918e-01,  6.9389e-01, -3.2649e-01,
          3.4802e-01,  2.2612e+00],
        [ 2.5643e-01,  2.5937e-01,  2.6746e-01,  1.7386e+00,  3.3593e+00,
         -1.6027e-01, -8.9792e-01],
        [-6.6302e-01, -6.6385e-01, -6.6059e-01, -6.6264e-01, -4.5806e-01,
          3.3380e-01, -7.2590e-01

In [ ]:
# Assuming you've already trained the model (model7) with the desired parameters

# Extract the last window from the training data
last_window = x_train[-window_size:]

# Print the last window
print("Last Window:")
print(last_window)

# Convert the last window to a PyTorch tensor
last_window_tensor = torch.tensor(last_window, dtype=torch.float32)

# Initialize hidden state
hidden = model7.init_hidden(batch_size=1)

# Forward pass to obtain the prediction for the next row
with torch.no_grad():
    prediction, _ = model7(last_window_tensor.unsqueeze(0), hidden)

# Convert the prediction to a numpy array
predicted_row = prediction.squeeze().numpy()

# Print or use the predicted row as needed
print("Predicted Row:")
print(predicted_row)

# Add the predicted row to the last window
next_window = np.vstack((last_window[1:], predicted_row))

# Print the next window
print("Next Window (including predicted row):")
print(next_window)


Last Window:
tensor([[-5.9760e-01, -5.8771e-01, -5.9358e-01, -4.2231e-01, -4.5338e-01,
          1.8267e+00,  7.7047e-01],
        [-7.3693e-01, -7.3899e-01, -7.3335e-01, -6.7598e-01, -4.5896e-01,
          8.7407e-01, -7.6385e-01],
        [-3.6628e-01, -3.7245e-01, -3.6745e-01,  2.6655e-01, -4.1044e-01,
         -6.6144e-01,  1.4002e+00],
        [-7.3982e-01, -7.4088e-01, -7.4107e-01,  2.2458e-01, -4.5467e-01,
         -5.6518e+00,  4.3665e-01],
        [-6.9224e-01, -6.9241e-01, -6.8822e-01, -6.5042e-01, -4.5190e-01,
          5.6484e-01, -2.9350e-01],
        [-5.4170e-01, -5.3405e-01, -5.3618e-01, -6.5565e-01, -4.5247e-01,
          8.1720e-01, -4.4281e-01],
        [ 4.1231e-01,  4.2911e-01,  4.0918e-01,  6.9389e-01, -3.2649e-01,
          3.4802e-01,  2.2612e+00],
        [ 2.5643e-01,  2.5937e-01,  2.6746e-01,  1.7386e+00,  3.3593e+00,
         -1.6027e-01, -8.9792e-01],
        [-6.6302e-01, -6.6385e-01, -6.6059e-01, -6.6264e-01, -4.5806e-01,
          3.3380e-01, -7.2590e-01

C:\Users\pitr7\AppData\Local\Temp\ipykernel_9756\1907871974.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  last_window_tensor = torch.tensor(last_window, dtype=torch.float32)


In [ ]:
# Assuming you've already trained the model (model7) with the desired parameters
# and you've already predicted the row (predicted_row)

# Extract the last window from the training data
last_window = x_train[-window_size:]

# Convert the last window to a PyTorch tensor
last_window_tensor = torch.tensor(last_window, dtype=torch.float32)

# Initialize hidden state
hidden = model7.init_hidden(batch_size=1)

# Forward pass to obtain the prediction for the next row
with torch.no_grad():
    prediction, _ = model7(last_window_tensor.unsqueeze(0), hidden)

# Convert the prediction to a numpy array
predicted_row = prediction.squeeze().numpy()

# Inverse transform the predicted row to its original scale
predicted_row_original_scale = scaler_y.inverse_transform(predicted_row.reshape(1, -1))

# Print the last window
print("Last Window:")
print(last_window)

# Print the predicted row in the original scale
print("Predicted Row (Original Scale):")
print(predicted_row_original_scale)

# Add the predicted row to the last window
next_window = np.vstack((last_window[1:], predicted_row))

# Print the next window
p


C:\Users\pitr7\AppData\Local\Temp\ipykernel_9756\3586926880.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  last_window_tensor = torch.tensor(last_window, dtype=torch.float32)


Last Window:
tensor([[-5.9760e-01, -5.8771e-01, -5.9358e-01, -4.2231e-01, -4.5338e-01,
          1.8267e+00,  7.7047e-01],
        [-7.3693e-01, -7.3899e-01, -7.3335e-01, -6.7598e-01, -4.5896e-01,
          8.7407e-01, -7.6385e-01],
        [-3.6628e-01, -3.7245e-01, -3.6745e-01,  2.6655e-01, -4.1044e-01,
         -6.6144e-01,  1.4002e+00],
        [-7.3982e-01, -7.4088e-01, -7.4107e-01,  2.2458e-01, -4.5467e-01,
         -5.6518e+00,  4.3665e-01],
        [-6.9224e-01, -6.9241e-01, -6.8822e-01, -6.5042e-01, -4.5190e-01,
          5.6484e-01, -2.9350e-01],
        [-5.4170e-01, -5.3405e-01, -5.3618e-01, -6.5565e-01, -4.5247e-01,
          8.1720e-01, -4.4281e-01],
        [ 4.1231e-01,  4.2911e-01,  4.0918e-01,  6.9389e-01, -3.2649e-01,
          3.4802e-01,  2.2612e+00],
        [ 2.5643e-01,  2.5937e-01,  2.6746e-01,  1.7386e+00,  3.3593e+00,
         -1.6027e-01, -8.9792e-01],
        [-6.6302e-01, -6.6385e-01, -6.6059e-01, -6.6264e-01, -4.5806e-01,
          3.3380e-01, -7.2590e-01

NameError: name 'p' is not defined

In [ ]:
ä

NameError: name 'ä' is not defined

In [ ]:

learning_rate = 0.001
window_size = 50
test_window_size = 50
hidden_dim = 64
n_layers = 4
batch_evaluation_frequency = 10
epochs = len(x_train) - window_size
batch_size = 1
input_size = x_train.shape[1]  # Input size based on your dataset
output_size = 1  # Output size (for regression task)

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_dim, n_layers, output_size):
        super(LSTMModel, self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_size, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_size)

    def forward(self, x, hidden):
        batch_size = x.size(0)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out[:, -1, :])
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                  weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        return hidden


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Instantiate the model
model = LSTMModel(input_size, hidden_dim, n_layers, output_size)

# Choose a loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

epochs = 1
window_size = 20

# Training loop
for epoch in range(epochs):
    for i in range(0, len(x_train) - window_size, batch_size):
        inputs = x_train[i:i+window_size]
        targets = y_train[i+window_size]

        # Convert inputs and targets to PyTorch tensors
        inputs = torch.tensor(inputs, dtype=torch.float32)
        targets = torch.tensor(targets, dtype=torch.float32)

        # Zero the gradients
        optimizer.zero_grad()

        # Initialize hidden state
        hidden = model.init_hidden(batch_size)

        # Forward pass
        outputs, _ = model(inputs.unsqueeze(0), hidden)
        loss = criterion(outputs.squeeze(), targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print loss or other metrics occasionally
        if i % batch_evaluation_frequency == 0:
            print(f'Epoch {epoch}, Batch {i}, Loss: {loss.item()}')




C:\Users\pitr7\AppData\Local\Temp\ipykernel_7768\3075939913.py:22: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = torch.tensor(inputs, dtype=torch.float32)
c:\Projectarbeit-Dow-Jones-Index\.venv\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Batch 0, Loss: 0.7257439494132996
Epoch 0, Batch 10, Loss: 0.42291468381881714
Epoch 0, Batch 20, Loss: 0.01817805878818035
Epoch 0, Batch 30, Loss: 1.215269923210144
Epoch 0, Batch 40, Loss: 2.999155044555664
Epoch 0, Batch 50, Loss: 0.6496577262878418
Epoch 0, Batch 60, Loss: 0.22550725936889648
Epoch 0, Batch 70, Loss: 1.995227336883545
Epoch 0, Batch 80, Loss: 0.3565322160720825
Epoch 0, Batch 90, Loss: 0.6110881567001343
Epoch 0, Batch 100, Loss: 0.5160682201385498
Epoch 0, Batch 110, Loss: 0.14027291536331177
Epoch 0, Batch 120, Loss: 0.0628763735294342
Epoch 0, Batch 130, Loss: 0.01936502568423748
Epoch 0, Batch 140, Loss: 0.0006627127877436578
Epoch 0, Batch 150, Loss: 0.13135549426078796
Epoch 0, Batch 160, Loss: 0.29241716861724854
Epoch 0, Batch 170, Loss: 3.9610371589660645
Epoch 0, Batch 180, Loss: 0.24971850216388702
Epoch 0, Batch 190, Loss: 0.22013649344444275
Epoch 0, Batch 200, Loss: 0.009745764546096325
Epoch 0, Batch 210, Loss: 11.56396770477295
Epoch 0, Ba

In [ ]:
# Get the feature names
feature_names = ['open', 'high', 'low', 'volume', 'adjusted_close', 'change_percent', 'avg_vol_20d']

# Print the last window with feature names
last_window_with_names = pd.DataFrame(last_window.numpy(), columns=feature_names)
print("Last Window with Feature Names:")
print(last_window_with_names)


Last Window with Feature Names:
        open      high       low    volume  adjusted_close  change_percent  \
0  -0.597599 -0.587705 -0.593579 -0.422313       -0.453383        1.826655   
1  -0.736933 -0.738988 -0.733346 -0.675977       -0.458957        0.874070   
2  -0.366276 -0.372448 -0.367455  0.266547       -0.410441       -0.661439   
3  -0.739821 -0.740883 -0.741068  0.224585       -0.454672       -5.651845   
4  -0.692244 -0.692407 -0.688216 -0.650423       -0.451903        0.564836   
5  -0.541696 -0.534049 -0.536179 -0.655645       -0.452472        0.817199   
6   0.412315  0.429114  0.409181  0.693892       -0.326494        0.348016   
7   0.256426  0.259366  0.267458  1.738583        3.359250       -0.160266   
8  -0.663018 -0.663852 -0.660589 -0.662645       -0.458059        0.333798   
9  -0.480950 -0.475760 -0.476806 -0.619113       -0.451480        0.138305   
10  3.424365  3.417197  3.450857 -0.316035       -0.000165       -0.096287   
11 -0.743007 -0.742694 -0.739481

In [ ]:
# Set the model to evaluation mode
model.eval()

# Use the last window for prediction
input_window = last_window.unsqueeze(0)  # Add batch dimension

# Initialize hidden state
hidden = model.init_hidden(1)  # Use batch_size = 1 for prediction

# Forward pass for prediction
predicted_output, _ = model(input_window, hidden)

# Convert the predicted output back to the original scale
predicted_output = scaler_y.inverse_transform(predicted_output.detach().numpy())

# Extract the last (window_size-1) values from the last window
last_window_values = last_window.numpy()[1:]

# Append the predicted value to the last window
next_window = np.concatenate((last_window_values, predicted_output))

# Reshape the next window to match the original shape
next_window = next_window.reshape(1, -1)

# Print the next window with feature names
next_window_with_names = pd.DataFrame(next_window, columns=feature_names)
print("Next Window (including the predicted row) with Feature Names:")
print(next_window_with_names)


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 7 and the array at index 1 has size 1